In [ ]:
from src.sim_utils.sim import Team, GameState, Player
import pandas as pd
#import yaml
#import matplotlib.pyplot as plt

In [2]:
from utils.quack import Quack

schedule = Quack.query('select season, week, away_team, home_team, ifnull(wind, 3) as wind, ifnull(temp,75) as temp, ifnull(spread_line,0) as spread_line from schedule where season = 2024 and week between 0 and 10')
schedule

,season,week,away_team,home_team,wind,temp,spread_line
0,2024,1,BAL,KC,8,67,3.0
1,2024,1,GB,PHI,3,75,2.0
2,2024,1,PIT,ATL,3,75,4.0
3,2024,1,ARI,BUF,20,61,6.5
4,2024,1,TEN,CHI,8,67,4.0
...,...,...,...,...,...,...,...
147,2024,10,TEN,LAC,3,75,8.0
148,2024,10,NYJ,ARI,3,75,-2.0
149,2024,10,PHI,DAL,3,75,-7.5
150,2024,10,DET,HOU,3,75,-4.5


In [4]:

vik = Team("DET", 2024, 15)
det = Team("CHI", 2024, 15)
game = GameState(vik, det, **{"wind": 5, "temp": 90, "spread_line": -12.5})
game.start_game()
game.play_game()
game.game_results(df=True).groupby(["team"]).sum()[
	["receiving_yards", "rushing_yards"]
]


TypeError: GameState.log_play() missing 1 required positional argument: 'play_time_elapsed'

In [43]:
df = Quack.query("""with agg as (select game_id, week, season, sum(carries), sum(attempts), sum(carries + attempts)/2 as plays from weekly where season = 2024 and week between 0 and 10 group by all order by 6)
                 select avg(plays) from agg
                 
                 """)
df

,avg(plays)
0,59.384868


In [53]:
game.game_results(df=True)[['completions','carries','team']].groupby('team').sum()

,completions,carries
team,,
CHI,15,74
DET,4,13


In [57]:
pd.DataFrame(game.pbp).iloc[0:10]

,play_type,yards_gained,player,possession,quarter,down,is_first_down,ydstogo,goal_to_go,yardline_100,...,wind,temp,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,drive,spread_line,total_line,posteam_timeouts_remaining,defteam_timeouts_remaining
0,kickoff,0.0,None,DET,1,1,True,10,0,65,...,5,90,900.000000,1800.000000,3600.000000,1,12.5,42,3,3
1,kickoff,0.0,None,DET,1,1,True,10,0,65,...,5,90,900.000000,1800.000000,3600.000000,1,12.5,42,3,3
2,run,NaN,David Montgomery,DET,1,1,True,10,0,65,...,5,90,900.000000,1800.000000,3600.000000,1,12.5,42,3,3
3,pass,NaN,David Montgomery,DET,1,1,True,10,0,65,...,5,90,892.060535,1792.060535,3592.060535,1,12.5,42,3,3
4,pass,NaN,David Montgomery,DET,1,2,False,10,0,65,...,5,90,893.837006,1793.837006,3593.837006,1,12.5,42,3,3
5,pass,NaN,David Montgomery,DET,1,3,False,10,0,65,...,5,90,864.728600,1764.728600,3564.728600,1,12.5,42,3,3
6,punt,NaN,None,DET,1,4,False,10,0,65,...,5,90,835.620193,1735.620193,3535.620193,1,12.5,42,3,3
7,run,NaN,D'Andre Swift,CHI,1,1,True,10,0,86,...,5,90,807.254162,1707.254162,3507.254162,2,-12.5,42,3,3
8,run,NaN,Caleb Williams,CHI,1,1,True,10,0,86,...,5,90,782.555847,1682.555847,3482.555847,2,-12.5,42,3,3
9,pass,NaN,Caleb Williams,CHI,1,1,True,10,0,86,...,5,90,749.787632,1649.787632,3449.787632,2,-12.5,42,3,3


In [8]:
small = schedule.loc[(schedule.home_team == 'MIA') | (schedule.away_team == 'MIA')]
for _, row in small.iterrows():
	try:
		team1 = Team(row['home_team'], row['season'], row['week'])
		team2 = Team(row['away_team'], row['season'], row['week'])
		game = GameState(team1, team2, **{'wind':row['wind'], 'temp':row['temp'], 'spread_line':row['spread_line']})
		print(team1,team2)
		#stats += log_sim(team1,team2,5)
	except Exception as e:
		print(team1.name, team2.name, row['season'],row['week'],e)

MIA has 0 points JAX has 0 points
MIA has 0 points BUF has 0 points
SEA has 0 points MIA has 0 points
MIA has 0 points TEN has 0 points
No data exists for NE in 2024
MIA TEN 2024 5 list index out of range
IND has 0 points MIA has 0 points
No data exists for MIA in 2024
IND MIA 2024 8 list index out of range
BUF has 0 points MIA has 0 points
LA has 0 points MIA has 0 points


In [25]:
def log_sim(team1,team2, n=1,**kwargs):
	result = []
	game = GameState(team1, team2, **kwargs)
	for _ in range(n):
		print(_)
		game.start_game()
		game.play_game()
		result += game.game_results()
		game.reset_game()
	return result

stats = []
for _, row in schedule.iterrows():
	try:
		team1 = Team(row['home_team'], row['season'], row['week'])
		team2 = Team(row['away_team'], row['season'], row['week'])
		print(team1,team2)
		stats += log_sim(team1,team2, n=1, wind=row['wind'], temp=row['temp'], spread_line=row['spread_line'])
	except Exception as e:
		print(team1.name, team2.name, row['season'],row['week'],e)

	

KC has 0 points BAL has 0 points
0
KC has won 17 - 14
PHI has 0 points GB has 0 points
0
GB has won 13 - 10
ATL has 0 points PIT has 0 points
0
PIT has won 14 - 7
BUF has 0 points ARI has 0 points
0
ARI has won 21 - 10
CHI has 0 points TEN has 0 points
0
TEN has won 3 - 0
No data exists for CIN in 2024
CHI TEN 2024 1 list index out of range
No data exists for IND in 2024
CHI TEN 2024 1 list index out of range
MIA has 0 points JAX has 0 points
0
MIA has won 10 - 3
No data exists for CAR in 2024
NO JAX 2024 1 list index out of range
NYG has 0 points MIN has 0 points
0
NYG has won 10 - 3
LAC has 0 points LV has 0 points
0
LAC has won 10 - 3
SEA has 0 points DEN has 0 points
0
DEN has won 7 - 0
CLE has 0 points DAL has 0 points
0
CLE has won 10 - 3
No data exists for TB in 2024
CLE DAL 2024 1 list index out of range
DET has 0 points LA has 0 points
0
LA has won 27 - 10
SF has 0 points NYJ has 0 points
0
RB -1 SF You want a player that does not exist
RB -1 SF You want a player that does not

In [26]:
df = pd.DataFrame(stats)
df

,team,position,id,sim_id,name,completions,attempts,passing_yards,passing_tds,interceptions,...,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,air_yards,yac
0,BAL,QB,00-0034796,1760387037793,Lamar Jackson,19,25,193,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BAL,RB,00-0032764,1760387037793,Derrick Henry,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19
2,BAL,RB,00-0034975,1760387037793,Justice Hill,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2
3,BAL,RB,00-0034808,1760387037793,John,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BAL,WR,00-0039064,1760387037793,Zay Flowers,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,LA,WR,00-0036427,1760387082434,Tyler Johnson,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2188,LA,WR,00-0038359,1760387082434,Xavier Smith,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2189,LA,TE,00-0036244,1760387082434,Colby,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2190,LA,TE,00-0039074,1760387082434,Davis Allen,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
szn = Quack.query('select * from weekly where season=2025 and week between 0 and 10')
szn = szn.groupby(['team','week'])[['rushing_yards', 'carries','receptions','receiving_yards', 'passing_yards','completions','attempts']].sum().reset_index().groupby(['team']).mean().round(2).reset_index().sort_values(by='carries', ascending=False)
szn['ypc'] = szn['rushing_yards'] / szn['carries']
szn['yp_pass'] = szn['receiving_yards'] / szn['attempts']
szn['plays'] = szn['attempts'] + szn['carries']
szn.sort_values(by='plays')


,team,week,rushing_yards,carries,receptions,receiving_yards,passing_yards,completions,attempts,ypc,yp_pass,plays
2,BAL,3.5,126.17,23.67,18.67,206.67,206.67,18.67,27.17,5.330376,7.606551,50.84
19,MIA,3.5,91.33,19.67,22.17,208.83,208.83,22.17,31.83,4.643111,6.560792,51.50
6,CIN,3.5,56.67,18.00,21.67,194.17,194.17,21.67,34.17,3.148333,5.682470,52.17
26,PIT,3.2,84.00,24.80,19.40,206.60,206.60,19.40,28.00,3.387097,7.378571,52.80
20,MIN,3.0,106.80,23.80,19.20,224.20,224.20,19.20,29.20,4.487395,7.678082,53.00
30,TEN,3.5,81.83,21.17,18.50,183.50,183.50,18.50,33.67,3.865376,5.449955,54.84
27,SEA,3.5,104.17,28.17,19.33,259.33,259.33,19.33,27.33,3.697906,9.488840,55.50
24,NYJ,3.5,135.67,27.33,18.83,175.33,175.33,18.83,28.33,4.964142,6.188846,55.66
25,PHI,3.5,95.33,27.00,19.50,195.33,195.33,19.50,28.67,3.530741,6.813045,55.67
31,WAS,3.0,156.40,26.60,17.80,205.40,205.40,17.80,29.20,5.879699,7.034247,55.80


In [28]:
simszn = df.groupby(['team','sim_id'])[['rushing_yards', 'carries','receptions','receiving_yards', 'passing_yards','completions','attempts']].sum().reset_index().groupby(['team']).mean().round(2).reset_index().sort_values(by='carries', ascending=False)
simszn['ypc'] = simszn['rushing_yards'] / simszn['carries']
simszn['yp_pass'] = simszn['receiving_yards'] / simszn['attempts']
simszn['plays'] = simszn['attempts'] + simszn['carries']
simszn.sort_values(by='attempts')

,team,sim_id,rushing_yards,carries,receptions,receiving_yards,passing_yards,completions,attempts,ypc,yp_pass,plays
14,KC,1.760387e+12,85.20,28.80,15.00,146.00,146.00,15.00,19.40,2.958333,7.525773,48.20
1,ATL,1.760387e+12,138.71,40.00,14.71,165.14,165.14,14.71,20.14,3.467750,8.199603,60.14
28,TB,1.760387e+12,106.00,43.00,15.67,177.00,177.00,15.67,21.00,2.465116,8.428571,64.00
12,IND,1.760387e+12,184.33,46.33,14.17,133.67,133.67,14.17,21.00,3.978632,6.365238,67.33
2,BAL,1.760387e+12,156.88,52.88,15.25,165.12,165.12,15.25,21.25,2.966717,7.770353,74.13
23,NYJ,1.760387e+12,114.30,37.40,13.60,143.40,143.40,13.60,21.30,3.056150,6.732394,58.70
8,DEN,1.760387e+12,201.83,59.00,14.17,163.17,163.17,14.17,21.50,3.420847,7.589302,80.50
18,MIA,1.760387e+12,133.00,43.00,15.67,175.67,175.67,15.67,21.67,3.093023,8.106599,64.67
25,PIT,1.760387e+12,199.12,58.62,16.12,175.38,175.38,16.12,21.75,3.396793,8.063448,80.37
29,TEN,1.760387e+12,144.83,46.17,14.33,160.67,160.67,14.33,21.83,3.136885,7.360055,68.00


In [13]:
def compare_stats(df, stats, season=2024, start_week=0, end_week=10, min_filter=None):
    """
    Query real avg stats from DB, compute median predicted stats, and compare them.
    
    quack: DB connection with .query()
    df: DataFrame with per-game predicted stats (includes 'name','team', and stat columns)
    stats: list of stat names (e.g. ['receptions','receiving_yards'])
    min_filter: optional dict to filter rows before mean, e.g. {'receptions': 2}
    """
    # --- build SQL dynamically ---
    select_cols = ", ".join([f"avg({s}) as {s}" for s in stats])
    sql = f"""
        select player_display_name as name, team, {select_cols}
        from weekly
        where season = {season} and week between {start_week} and {end_week}
        group by all
    """
    real = Quack.query(sql)

    # --- median predicted stats ---
    med = (
        df.groupby(['name', 'team'])[stats]
        .median()
        .round(2)
        .reset_index()
    )

    # --- merge + compare ---
    comb = real.merge(med, on=['name', 'team'], suffixes=('_real', '_pred'))
    results = {}
    for s in stats:
        res_col = f"{s}_res"
        comb[res_col] = comb[f"{s}_real"] > comb[f"{s}_pred"]
        tmp = comb
        if min_filter and s in min_filter:
            tmp = tmp[tmp[f"{s}_pred"] > min_filter[s]]
        results[s] = tmp[res_col].mean()
    return results

compare_stats(df, ['receptions','carries','receiving_yards','rushing_yards', 'passing_yards'], min_filter=[2,2,20,20,40])

{'receptions': np.float64(0.6512820512820513),
 'carries': np.float64(0.30512820512820515),
 'receiving_yards': np.float64(0.7025641025641025),
 'rushing_yards': np.float64(0.3282051282051282),
 'passing_yards': np.float64(0.11538461538461539)}

In [76]:
Quack.query('select player_name, player_display_name from weekly')

,player_name,player_display_name
0,S.Morris,Sylvester Morris
1,C.Stanley,Chad Stanley
2,S.Morris,Sylvester Morris
3,C.McNown,Cade McNown
4,T.McBride,Tod McBride
...,...,...
419039,J.Flacco,Joe Flacco
419040,A.Rodgers,Aaron Rodgers
419041,T.Way,Tress Way
419042,N.Folk,Nick Folk


In [ ]:
df = game.game_results(df=True)
df[['sim_id','team', 'position', 'name',
	'carries', 'rushing_yards',       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
	'rushing_tds', 
	'rushing_first_downs',  'rushing_2pt_conversions',

	'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
	'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
	'receiving_2pt_conversions', 'target_share', 'air_yards_share',
	'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',
	'air_yards', 'yac']]

,sim_id,team,position,name,carries,rushing_yards,receptions,targets,receiving_yards,receiving_tds,...,receiving_epa,receiving_2pt_conversions,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,air_yards,yac
0,1760305815671,CHI,QB,Caleb Williams,2,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1760305815671,CHI,QB,Tyson Bagent,4,15,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1760305815671,CHI,RB,D'Andre Swift,77,263,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1760305815671,CHI,RB,Travis Homer,6,24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1760305815671,CHI,WR,Rome Odunze,1,2,4,7,52,0,...,0,0,0,0,0,0,0,0,33,19
5,1760305815671,CHI,WR,Keenan Allen,0,0,4,7,102,0,...,0,0,0,0,0,0,0,0,52,50
6,1760305815671,CHI,WR,DJ Moore,0,0,8,11,90,0,...,0,0,0,0,0,0,0,0,44,46
7,1760305815671,CHI,WR,Tyler Scott,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1760305815671,CHI,TE,Cole Kmet,0,0,2,2,12,0,...,0,0,0,0,0,0,0,0,6,6
9,1760305815671,DET,QB,Jared Goff,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df = (
	game.game_results(df=True)
	.groupby(["team_name"])
	.sum()[["receiving_yards", "rushing_yards", "carries"]]
)
df["ypc"] = df["rushing_yards"] / df["carries"]
df

,receiving_yards,rushing_yards,carries,ypc
team_name,,,,
CHI,409,276,92,3.0
DET,260,12,5,2.4


In [ ]:
def run_sim(team1, team2, num_sims, config=config):
	results = []
	scores = []
	vik = Team(team1, 2024, 13)
	det = Team(team2, 2024, 13)
	game = GameState(vik, det, config, **{"wind": 5, "temp": 60, "spread_line": 6.5})
	for i in range(num_sims):
		game.start_game()
		game.play_game()
		scores.append((game.home.score, game.away.score))
		res = vik.game_results() + det.game_results()
		for result in res:
			result["game_number"] = i
		results += res
	return results, scores


results, scores = run_sim(
	"DET", "CHI", 10
)  # 28.8 -> 22.8 by pulling game initialization out of loop

DET has won 20 - 13
CHI has won 35 - 21
DET has won 35 - 20
CHI has won 28 - 20
CHI has won 35 - 24
DET has won 14 - 9
CHI has won 24 - 20
DET has won 27 - 14
DET has won 28 - 21
DET has won 42 - 20


In [ ]:
df = pd.json_normalize(results)
df.groupby(["name", "id"])[
	[
		"receiving_yards",
		"targets",
		"receptions",
		"air_yards",
		"completions",
		"attempts",
		"rushing_yards",
		"carries",
	]
].agg(["median", "mean"]).sort_values(by=[("receiving_yards", "mean")], ascending=False)

receiving_yards       targets      receptions  \
                                      median  mean  median mean     median   
name              id                                                         
Sam LaPorta       00-0039065            74.5  88.4     8.5  8.9        5.5   
Amon-Ra St. Brown 00-0036963            86.0  87.0     8.0  7.9        4.5   
Jahmyr Gibbs      00-0039139            66.0  78.4     7.0  6.8        4.5   
Cole Kmet         00-0036290            38.5  60.6     5.0  5.3        2.5   
Rome Odunze       00-0039919            52.5  56.9     4.5  4.9        2.5   
D.J. Moore        00-0034827            40.5  50.5     6.5  6.7        3.0   
Keenan Allen      00-0030279            40.0  40.4     5.5  6.5        3.0   
Tim Patrick       00-0033375            15.5  32.3     3.0  3.1        1.5   
D'Andre Swift     00-0036275            30.0  30.3     4.0  4.2        2.5   
David Montgomery  00-0035685            19.5  29.4     4.5  4.2        3.0   
Allen Robinson    00-0031428             0.5  17.0     1.0  1.3        0.5   
Marcedes Lewis    00-0024243             1.5   9.9     0.5  0.8        0.5   
Brock Wright      00-0036754             0.0   9.4     1.5  1.4        1.0   
Roschon Johnson   00-0039021             0.0   8.7     1.0  1.2        0.0   
Gerald Everett    00-0033895             0.0   6.2     0.0  0.4        0.0   
DeAndre Carter    00-0031763             0.0   0.8     0.5  0.6        0.0   
Caleb Williams    00-0039918             0.0   0.0     0.0  0.0        0.0   
Jared Goff        00-0033106             0.0   0.0     0.0  0.0        0.0   
Craig Reynolds    00-0035567             0.0   0.0     0.0  0.0        0.0   
Sione Vaki        00-0039364             0.0   0.0     0.0  0.0        0.0   
Travis Homer      00-0035594             0.0   0.0     0.0  0.0        0.0   

                                  air_yards       completions       attempts  \
                             mean    median  mean      median  mean   median   
name              id                                                           
Sam LaPorta       00-0039065  5.5      38.0  49.0         0.0   0.0      0.0   
Amon-Ra St. Brown 00-0036963  4.4      30.0  44.1         0.0   0.0      0.0   
Jahmyr Gibbs      00-0039139  4.6       9.5  15.3         0.0   0.0      0.0   
Cole Kmet         00-0036290  3.1      18.5  35.0         0.0   0.0      0.0   
Rome Odunze       00-0039919  3.0      23.0  36.3         0.0   0.0      0.0   
D.J. Moore        00-0034827  3.3       8.0  11.8         0.0   0.0      0.0   
Keenan Allen      00-0030279  2.9       5.0  18.6         0.0   0.0      0.0   
Tim Patrick       00-0033375  1.5       4.0  17.1         0.0   0.0      0.0   
D'Andre Swift     00-0036275  2.4       7.5  11.0         0.0   0.0      0.0   
David Montgomery  00-0035685  2.7      -3.5  -0.8         0.0   0.0      0.0   
Allen Robinson    00-0031428  0.8       0.0   7.4         0.0   0.0      0.0   
Marcedes Lewis    00-0024243  0.6       0.0   2.0         0.0   0.0      0.0   
Brock Wright      00-0036754  0.7       0.0   1.8         0.0   0.0      0.0   
Roschon Johnson   00-0039021  0.5       0.0   0.4         0.0   0.0      0.0   
Gerald Everett    00-0033895  0.4       0.0   4.4         0.0   0.0      0.0   
DeAndre Carter    00-0031763  0.2       0.0   0.2         0.0   0.0      0.0   
Caleb Williams    00-0039918  0.0       0.0   0.0        16.0  16.4     30.5   
Jared Goff        00-0033106  0.0       0.0   0.0        18.0  20.2     32.5   
Craig Reynolds    00-0035567  0.0       0.0   0.0         0.0   0.0      0.0   
Sione Vaki        00-0039364  0.0       0.0   0.0         0.0   0.0      0.0   
Travis Homer      00-0035594  0.0       0.0   0.0         0.0   0.0      0.0   

                                   rushing_yards       carries        
                              mean        median  mean  median  mean  
name              id                                                  
Sam LaPorta       00-0

In [ ]:
df.groupby(["team_name", "game_number"])[
	[
		"receiving_yards",
		"targets",
		"receptions",
		"air_yards",
		"completions",
		"attempts",
		"rushing_yards",
		"carries",
	]
].sum().groupby(["team_name"]).mean().sort_values(
	by=["receiving_yards"], ascending=False
)

,receiving_yards,targets,receptions,air_yards,completions,attempts,rushing_yards,carries
team_name,,,,,,,,
CAR,393.4,34.9,22.3,190.7,22.3,34.9,142.9,27.7
PHI,340.0,33.2,19.7,177.1,19.7,33.2,164.4,35.5


In [ ]:
sched = pd.read_csv("week_13.csv", index_col=0)
sched.iloc[11]

game_id           2024_13_TB_CAR
season                      2024
week                          13
home_team                    CAR
away_team                     TB
home_moneyline             205.0
spread_line                 -6.5
total_line                  46.5
temp                        50.0
wind                         8.0
Name: 6896, dtype: object

In [ ]:
def run_sim(num_sims, sched, config=config):
	results = []
	scores = []
	for i in range(num_sims):
		vik = Team(sched["home_team"], sched["season"], sched["week"])
		det = Team(sched["away_team"], sched["season"], sched["week"])
		game = GameState(vik, det, config, **sched)
		game.start_game()
		game.play_game()
		scores.append((game.home.score, game.away.score))
		res = vik.game_results() + det.game_results()
		for result in res:
			result["game_number"] = i
		results += res
	return results, scores


# results, scores = run_sim('KC','LV',100)

In [ ]:
results = []
test = True
if test:
	for t in [11]:
		# for t in range(len(sched)):
		game1 = sched.iloc[t].to_dict()
		try:
			results, scores = run_sim("KC", "LV", 100, game1)
		except Exception as _:
			print("oops")

CAR has won 13 - 7
CAR has won 31 - 7
CAR has won 30 - 6
CAR has won 26 - 17
TB has won 28 - 24
CAR has won 34 - 27
TB has won 24 - 14
TB has won 27 - 20
TB has won 28 - 24
CAR has won 24 - 20
CAR has won 17 - 13
CAR has won 21 - 13
CAR has won 40 - 14
CAR has won 24 - 14
TB has won 16 - 16
TB has won 20 - 19
TB has won 23 - 21
CAR has won 26 - 17
TB has won 21 - 17
TB has won 26 - 17
TB has won 17 - 17
CAR has won 13 - 10
CAR has won 24 - 9
CAR has won 45 - 17
CAR has won 44 - 17
TB has won 31 - 23
CAR has won 31 - 21
TB has won 20 - 7
TB has won 17 - 17
CAR has won 23 - 17
TB has won 27 - 6
CAR has won 28 - 27
TB has won 21 - 16
TB has won 24 - 6
TB has won 24 - 14
CAR has won 41 - 16
CAR has won 23 - 17
CAR has won 41 - 38
TB has won 31 - 14
TB has won 20 - 17
TB has won 34 - 14
CAR has won 20 - 10
TB has won 10 - 10
TB has won 31 - 10
CAR has won 34 - 17
TB has won 24 - 23
TB has won 21 - 7
TB has won 26 - 26
CAR has won 41 - 16
CAR has won 20 - 9
TB has won 24 - 24
CAR has won 24 

In [ ]:
weekly = pd.read_csv("data/weekly.csv", index_col=0)
pit = weekly.loc[
	(weekly.week == 12)
	& (weekly.season == 2024)
	& ((weekly.recent_team == "PIT") | (weekly.recent_team == "CLE"))
]